In [1]:
!pip install PyDrive
import os
import json
import datetime
import pytz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1cZ4NMEAMD0Kd64wnv9epbnofDipFrNRd'})
download.GetContentFile('ECE219_tweet_data.zip')

In [4]:
!unzip ECE219_tweet_data.zip

Archive:  ECE219_tweet_data.zip
  inflating: tweets_#nfl.txt         
  inflating: tweets_#superbowl.txt   
  inflating: tweets_#sb49.txt        
  inflating: tweets_#patriots.txt    
  inflating: tweets_#gohawks.txt     
  inflating: tweets_#gopatriots.txt  


In [0]:
download = drive.CreateFile({'id': '1RvxD3Yx76NWqm5QAg_0gIwUt2foQu8T3'})
download.GetContentFile('ECE219_tweet_test.zip')

In [6]:
!unzip ECE219_tweet_test.zip

Archive:  ECE219_tweet_test.zip
  inflating: sample2_period1.txt     
  inflating: sample2_period3.txt     
  inflating: sample2_period2.txt     
  inflating: sample1_period2.txt     
  inflating: sample1_period3.txt     
  inflating: sample1_period1.txt     
  inflating: sample0_period1.txt     
  inflating: sample0_period3.txt     
  inflating: sample0_period2.txt     


In [0]:
import numpy as np 
np.random.seed(42) 
import random 
random.seed(42)

In [8]:
path = "./"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]
topics = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
test_files_1 =["sample0_period1.txt","sample0_period2.txt","sample0_period3.txt"]
test_files_2 =["sample1_period1.txt","sample1_period2.txt","sample1_period3.txt"]
test_files_3 =["sample2_period1.txt","sample2_period2.txt","sample2_period3.txt"]
for i, fl in enumerate(files):
    print("files[" + str(i) + "] => " + fl)


files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [0]:

def transfer_time(data_raw,time_type):
    
    pst_tz = pytz.timezone('America/Los_Angeles')
    
    # sort according to time
    pddata_raw = pd.DataFrame(data_raw,columns=['time','tweets','retweets','followers','mentioned', 'media','active','author','favourites_count','title'])
    pddata_raw = pddata_raw.sort_values(by = 'time')
    pddata_raw = pddata_raw.reset_index(drop=True)               

    # calculate hour index and minute index from time
    if time_type == 'hour':
        hour_accu = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)  
            hour_accu.append(((p.month-1)*31+p.day-14)*24+p.hour)                             
        pddata_raw['time'] = hour_accu
    elif time_type == 'minute':
        minu_accu = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)                    
            minu_accu.append((((p.month-1)*31+p.day-14)*24 + (p.hour-0))*12 + p.minute//5)             
        pddata_raw['time'] = minu_accu    
    else:
        print("Invalid time type")
        
    return pddata_raw

In [0]:
def generate_df(pddata_raw):
    df = pd.DataFrame([],columns=['time unit','tweets','retweets','followers sum','followers max',\
                                  'mentioned','media','active','author','favourites_count','title'])
    
    col = pddata_raw.columns.get_loc('time')
    df['time unit'] = range(int(pddata_raw.iloc[len(pddata_raw.index)-1,col] - pddata_raw.iloc[0,col]+1))

    df['tweets'] = pddata_raw.groupby("time")['tweets'].sum()
    df['retweets'] = pddata_raw.groupby("time")['retweets'].sum()
    df['followers sum'] = pddata_raw.groupby("time")['followers'].sum()
    df['followers max'] = pddata_raw.groupby("time")["followers"].max()
    df['mentioned'] = pddata_raw.groupby("time")['mentioned'].sum()
    df['media'] = pddata_raw.groupby("time")['media'].sum()
    df['active'] = pddata_raw.groupby("time")['active'].mean()  
    df['author'] = pddata_raw.groupby("time")['author'].nunique() # count number of not-repeating authors    
    df['favourites_count'] = pddata_raw.groupby("time")['favourites_count'].sum()
    df['title'] = pddata_raw.groupby("time")['title'].mean()
            
    # reset index of df
    df = df.fillna(0).reset_index(drop=True)
    
    return df

In [0]:
import time
def parse_dataset(files):
    start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
    end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))   

    start_hour_idx = ((2-1)*31+1-14)*24+8
    end_hour_idx = ((2-1)*31+1-14)*24+20
    start_minute_idx = (((2-1)*31+1-14)*24 + (8-0))*12 + 0//5    
        
    # extract raw features
    data_raw = [[],[],[]]
    for file in files:
        for line in open(path + file, 'r', encoding="utf-8") :
            row_tmp = []
            a = json.loads(line)
            citation_date = a['citation_date']
            tweet = 1
            retweet = a['metrics']['citations']['total']
            foll = a['author']['followers']             
            ment = len(a['tweet']['entities']['user_mentions'])        
            medi = len(a['tweet']['extended_entities']['media']) if 'extended_entities' in a['tweet'] else 0
            hist_tw = a['tweet']['user']["statuses_count"]
            hist_yr = a['tweet']['user']['created_at'][-4:]
            acti = hist_tw/(2015-float(hist_yr)+1) 
            auth = a['author']['name']
            favo = a['tweet']['user']['favourites_count']
            titl = len(a['title'])

            # append to row_tmp
            row_tmp.append(citation_date)        
            row_tmp.append(tweet)        
            row_tmp.append(retweet)
            row_tmp.append(foll)    
            row_tmp.append(ment) 
            row_tmp.append(medi) 
            row_tmp.append(acti)  
            row_tmp.append(auth)
            row_tmp.append(favo)
            row_tmp.append(titl)

            # assign to 3 periods
            if citation_date < start_time:
                data_raw[0].append(row_tmp)
            elif citation_date < end_time:
                data_raw[1].append(row_tmp)
            else:
                data_raw[2].append(row_tmp)            

    # generate raw pandas dataframe
    pddata_raw_1 = transfer_time(data_raw[0],'hour')
    pddata_raw_2 = transfer_time(data_raw[1],'minute')
    pddata_raw_2['time'] = pddata_raw_2['time'] - start_minute_idx
    pddata_raw_3 = transfer_time(data_raw[2],'hour')
    pddata_raw_3['time'] = pddata_raw_3['time'] - end_hour_idx - 1    
            
    # generate df and df_y for each time slot
    df_1 = generate_df(pddata_raw_1)  
    df_y_1 = df_1.iloc[1:,1].reset_index(drop=True)
    df_1 = df_1[:len(df_y_1)]
    
    df_2 = generate_df(pddata_raw_2)
    df_y_2 = df_2.iloc[1:,1].reset_index(drop=True)
    df_2 = df_2[:len(df_y_2)]
   
    df_3 = generate_df(pddata_raw_3)
    df_y_3 = df_3.iloc[1:,1].reset_index(drop=True)
    df_3 = df_3[:len(df_y_3)]
    
    return (df_1.iloc[:,1:],df_y_1), (df_2.iloc[:,1:],df_y_2), (df_3.iloc[:,1:],df_y_3)

In [0]:
def plot_recipe(df_y, pred_y):
    plt.figure()
    area = np.pi * (4)**2/4
    plt.scatter(df_y, pred_y, s = area)
    plt.plot([df_y.min(), df_y.max()], [df_y.min(), df_y.max()], 'k--', lw = 1)
    plt.xlabel('true values')
    plt.ylabel('fitted values')
    plt.show()

In [0]:
import statsmodels.api as sm
def ols_regression(df, df_y):
    X2 = sm.add_constant(df)
    y = df_y.as_matrix()
    lm = sm.OLS(y, X2).fit()
    print(lm.summary())
    print(list(df))

In [0]:
(df,df_y),(df_2,df_y_2),(df_3,df_y_3) = parse_dataset(files)

In [0]:
from sklearn.model_selection import KFold
param_grid={
'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
'max_features': ['auto', 'sqrt'],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [200, 400, 600, 800, 1000,
1200, 1400, 1600, 1800, 2000]
}
kf = KFold(n_splits=5,random_state=42,shuffle=True)

In [16]:
df_3.head()

,tweets,retweets,followers sum,followers max,mentioned,media,active,author,favourites_count,title
0,6492.0,52858.0,106280297.0,15997691.0,3689.0,1919.0,3527.647824,4568.0,15634832.0,101.818392
1,9037.0,48221.0,61182588.0,9675466.0,5516.0,2340.0,5242.472635,5732.0,17620137.0,103.982406
2,5018.0,16778.0,65565526.0,9675466.0,2313.0,1166.0,5628.756802,3186.0,8980699.0,103.580311
3,3292.0,14989.0,35632346.0,5025007.0,1449.0,779.0,6170.050290,2044.0,5958408.0,106.211422
4,2605.0,5965.0,16128911.0,3093728.0,1342.0,528.0,6052.328881,1531.0,4116194.0,106.714012


In [17]:

df_y_2.head()

0    1336.0
1    1231.0
2    1299.0
3    1230.0
4     321.0
Name: tweets, dtype: float64

## GradientBoostingRegressor

In [21]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
avg_RMSE1_train=np.zeros((8,2,3,3,10))
avg_RMSE1_test=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df):
                        X_train= df.iloc[train_index]
                        y_train= df_y.iloc[train_index]
                        X_test= df.iloc[test_index]
                        y_test= df_y.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE1_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE1_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE1_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE1_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.004491047574769436 RMSE_test= 2217.451813076308

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.0003150023432260133 RMSE_test= 2217.4517507372975

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.0003150023432266519 RMSE_test= 2217.4517507372975

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.0003150023432265886 RMSE_test= 2217.4517507372975

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0003150023432266163 RMSE_test= 2217.4517507372975

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.00031500234322668047 RMSE_test= 2217.4517507372975

max_depth= 10 max_features= auto min_samples_leaf= 1 min_

In [22]:
print("min RMSE in testset=",np.min(avg_RMSE1_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE1_test), avg_RMSE1_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 1885.9909330327248
parameters:
max_depth= 40 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200



In [23]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df,df_y)
pred = reg.predict(df)
print("MSE=",mean_squared_error(df_y, pred))

MSE= 1.8094654232607348


In [24]:
avg_RMSE2_train=np.zeros((8,2,3,3,10))
avg_RMSE2_test=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df_2):
                        X_train= df_2.iloc[train_index]
                        y_train= df_y_2.iloc[train_index]
                        X_test= df_2.iloc[test_index]
                        y_test= df_y_2.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE2_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE2_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE2_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE2_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.0002976109947840484 RMSE_test= 1950.0041747238213

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.0002976109947843608 RMSE_test= 1950.0041747238213

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.0002976109947843608 RMSE_test= 1950.0041747238213

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.0002976109947843608 RMSE_test= 1950.0041747238213

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0002976109947843608 RMSE_test= 1950.0041747238213

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.0002976109947843608 RMSE_test= 1950.0041747238213

max_depth= 10 max_features= auto min_samples_leaf= 1 min

In [25]:
print("min RMSE in testset=",np.min(avg_RMSE2_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE2_test), avg_RMSE2_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 1753.3871782573021
parameters:
max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 400



In [26]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df_2,df_y_2)
pred = reg.predict(df_2)
print("MSE=",mean_squared_error(df_y_2, pred))

MSE= 9.891895423393578e-08


In [27]:
avg_RMSE3_train=np.zeros((8,2,3,3,10))
avg_RMSE3_test=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df_3):
                        X_train= df_3.iloc[train_index]
                        y_train= df_y_3.iloc[train_index]
                        X_test= df_3.iloc[test_index]
                        y_test= df_y_3.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE3_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE3_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE3_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE3_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.00031432078370507245 RMSE_test= 702.4223184051948

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.0003143207837194469 RMSE_test= 702.4223184051948

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.0003143207837219933 RMSE_test= 702.4223184051948

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.00031432078372289847 RMSE_test= 702.4223184051948

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.00031432078372326406 RMSE_test= 702.4223184051948

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.0003143207837234154 RMSE_test= 702.4223184051948

max_depth= 10 max_features= auto min_samples_leaf= 1 min_sa

In [28]:
print("min RMSE in testset=",np.min(avg_RMSE3_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE3_test), avg_RMSE3_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 564.1801697048221
parameters:
max_depth= 20 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 2000



In [29]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df_3,df_y_3)
pred = reg.predict(df_3)
print("MSE=",mean_squared_error(df_y_3, pred))

MSE= 9.990204162366395e-08
